In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import math
import torch.nn.functional as F

- scaled dot product attention
  - multi-head attention의 핵심적인 부분
  - query, key의 transpose의 matmul(dot product) -> similarity를 계산
  - 이 similarity를 기반으로 value값을 참조
  - decoder 부분에서는 masking처리를 해야함 -> if문으로 mask부분 포함해서 구현

In [9]:
def scaled_dot_product_attention(self, q, k, v, mask = None):
    d_k = k.size()[-1]
    k_transpose = torch.transpose(k, 3, 2)
    output = torch.matmul(q, k_transpose)
    output = output / math.sqrt(d_k)
    if mask is not None:
        output = output.masked_fill(mask.unsqueeze(1).unsqueeze(-1), 0)
    output = F.softmax(output, -1)
    output = torch.matmul(output, v)
    return output

- Multi-Head Attention
  - scaled dot-product attention을 query에 해당하는 value 값들을 참조하기 위해 사용
    - 이 때 query, key, value를 그대로 사용안함
    - 여러개의 head로 나누고 query, key, value를 linear projection한 후 사용
  - scaled dot-product attention 이후 각 head의 value값을 concat -> linear layer -> output
  - contiguous : sequence의 순서가 중요 -> 순서 유지를 위해 사용

In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim_num = 512, head_num = 8):
        super().__init__()
        self.head_num = head_num
        self.dim_num = dim_num
        self.query_embed = nn.Linear(dim_num, dim_num)
        self.key_embed = nn.Linear(dim_num, dim_num)
        self.value_embed = nn.Linear(dim_num, dim_num)
        self.output_embed = nn.Linear(dim_num, dim_num)

    def scaled_dot_product_attention(self, q, k, v, mask = None):
        d_k = k.size()[-1]
        k_transpose = torch.transpose(k, 3, 2)
        output = torch.matmul(q, k_transpose)
        output = output / math.sqrt(d_k)
        if mask is not None:
            output = output.masked_fill(mask.unsqueeze(1).unsqueeze(-1), 0)
        output = F.softmax(output, -1)
        output = torch.matmul(output, v)
        return output
    
    def forward(self, q, k, v, mask = None):
        batch_size = q.size()[0]
        q = self.query_embed(q).view(batch_size, -1, self.head_num, self.dim_num // self.head_num).transpose(1, 2)
        k = self.key_embed(k).view(batch_size, -1, self.head_num, self.dim_num // self.head_num).transpose(1, 2)
        v = self.value_embed(v).view(batch_size, -1, self.head_num, self.dim_num//self.head_num).transpose(1, 2)
        output = self.scaled_dot_product_attention(q, k, v, mask)
        batch_num, head_num, seq_num, hidden_num = output.size()
        output = torch.transpose(output, 1, 2).contiguous().view((batch_size, -1, hidden_num * self.head_num))
        return output

- Layer norm
  - dimension layer 방향으로 평균을 빼고 표준편차로 나누는 Normalization 기법

$$y = \frac{x - E[x]}{\sqrt{Var[x] + \epsilon}} * \gamma + \beta$$

In [11]:
def layer_norm(self, input):
    mean = torch.mean(input, dim = -1, keepdim = True)
    std = torch.std(input, dim = -1, keepdim = True)
    output = (input - mean) / std
    return output

# torch.nn.LayerNorm(normalized_shape, eps, elementwise_affine = True, bias = True, device = None, dtype = None)

- Add & LayerNorm
  - 이전 층의 output을 layer norm을 통해 normalization한 후 residual값을 더해줌

In [12]:
class AddLayerNorm(nn.Module):
    def __init__(self):
        super().__init__()
    
    def layer_norm(self, input):
        mean = torch.mean(input, dim = -1, keepdim = True)
        std = torch.std(input, dim = -1, keepdim = True)
        output = (input - mean) / std
        return output

    def forward(self, input, residual):
        return residual + self.layer_norm(input)

- Feed Forward
  - Fully Connected Layer -> ReLU -> Fully Connected Layer로 구성되어 있음

$$FFN(x) = max(0, xW_1 + b_1)W_2 + b_2$$

In [13]:
class FeedForward(nn.Module):
    def __init__(self, dim_num = 512):
        super().__init__()
        self.layer1 = nn.Linear(dim_num, dim_num *4)
        self.layer2 = nn.Linear(dim_num*4, dim_num)

    def forward(self, input):
        output = self.layer1(input)
        output = self.layer2(F.relu(output))
        return output

- Encoder
  - Multi-Head Attention -> Residual Add & Layer Norm -> Feed Forward -> Residual Add & Layer Norm
  - sublayer들을 연결하는 방식으로 구현

In [14]:
class Encoder(nn.Module):
    def __init__(self, dim_num = 512):
        super().__init__()
        self.multihead = MultiHeadAttention(dim_num = dim_num)
        self.addlayernorm = AddLayerNorm()
        self.feedforward = FeedForward(dim_num = dim_num)
        self.layer2 = AddLayerNorm()

    def forward(self, q, k, v):
        multihead_output = self.multihead(q, k, v)
        residual1_output = self.addlayernorm(multihead_output, q)
        feedforward_output = self.feedforward(residual1_output)
        output = self.layer2(feedforward_output, residual1_output)
        return output

- Decoder
  - Masked Multi-Head Attention -> Residual Add & Layer Norm -> Multi-Head Attention -> Residual Add & Layer Norm -> Feed Forward -> Residual Add & Layer Norm
  - Encoder와 비슷하게 sublayer들을 연결하는 방식으로 구현
    - 중간 Multi-Head Attention은 query, key를 Encoder의 Output을 사용 -> 이 점을 명시해야 함
  - masking을 사용

In [15]:
class Decoder(nn.Module):
    def __init__(self, dim_num = 512):
        super().__init__()
        self.masked_multihead = MultiHeadAttention(dim_num = dim_num)
        self.residual1 = AddLayerNorm()
        self.multihead = MultiHeadAttention(dim_num = dim_num)
        self.residual2 = AddLayerNorm()
        self.feed_forward = FeedForward(dim_num = dim_num)
        self.residual3 = AddLayerNorm()

    def forward(self, o_q, o_k, o_v, encoder_output, mask):
        masked_output = self.masked_multihead(o_q, o_k, o_v, mask)
        res1_output = self.residual1(masked_output, o_q)
        multihead_output = self.multihead(encoder_output, encoder_output, res1_output, mask)
        res2_output = self.residual2(multihead_output, res1_output)
        feedforward_output = self.feed_forward(res2_output)
        output = self.residual3(feedforward_output, res2_output)
        return output

- positional_encoding
  - 짝수번째 token과 홀수번째 token이 각기 다른 식을 따름
    - i = hidden dimension 방향의 인덱스, pos = postional 방향(몇 번째 seq인지)
  - 크게 두 부분에서 사용됨
    - input, output의 sequence length길이가 다를 수 있음
      - 이것을 인자로 받게 구성
  - self.register_buffer : model parameter 학습 시 positional encoding이 학습되지 않도록 막아주기 위한 용도

$$PE_{(pos, 2i)} = sin(pos/10000^{21/d_{model}}) \\ PE_{(pos, 2i+1)} = cos(pos/10000^{2i/d_{model}})$$

In [16]:
def positional_encoding(self, position_max_length = 100):
    position = torch.arange(0, position_max_length, dtype = torch.float).unsqueeze(1)
    pe = torch.zeros(position_max_length, self.hidden_dim)
    div_term = torch.pow(torch.ones(self.hidden_dim // 2).fill_(10000), torch.arange(0, self.hidden_dim, 2) / torch.tensor(self.hidden_dim, dtype = torch.float32))
    pe[:, 0::2] = torch.sin(position/div_term)
    pe[:, 1::2] = torch.cos(position / div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)

    return pe

- input & output embedding
  - nn.Embedding을 통해 쉽게 구현 가능
  - 첫번째 인자 : input 데이터의 total word 개수, 두 번째 인자는 hidden dimension의 수
  - total_word_num : sequence dictionary에 존재하는 unique value의 개수(전체 단어 개수 아님)
    - task에 따라 input, output의 단어의 개수가 다를 수 있어 다른 인자를 받아야함(번역)

In [17]:
# self.input_data_embedding = nn.Embedding(total_word_num, self.hidden_dim)
# self.output_data_embedding = nn.Embedding(total_word_num, self.hidden_dim)

- Transformer
  - Encoder 부분은 앞서 구현했던 Encoder를 n번 반복
  - Encoder 부분에 들어가는 query, key, value는 문장의 embedding한 값으로 모두 같고, 전번째 encoder의 결과가 다음 encoder의 query, key, value가 됨
  - Decoder 부분도 비슷
    - Encoder의 output이 사용 된다는 점, Decoder에서는 다음 sequence를 볼 수 없음 -> mask를 사용해서
    - 이 두가지 차이가 존재
  - 실제 학습에서 Decoder에 masking을 매우 작은 값을 사용하는게 유리함

In [18]:
class Transformer(nn.Module):
    def __init__(self, encoder_num = 6, decoder_num = 6, hidden_dim = 512, max_encoder_seq_len = 100, max_decoder_seq_len = 100):
        super().__init__()
        self.encoder_num = encoder_num
        self.hidden_dim = hidden_dim
        self.decoder_num = decoder_num
        self.max_encoder_seq_len = max_encoder_seq_len
        self.max_decoder_seq_len = max_decoder_seq_len
        self.input_data_embed = nn.Embedding(max_encoder_seq_len, self.hidden_dim)
        self.Encoders = [Encoder(dim_num = hidden_dim) for _ in range(encoder_num)]
        self.output_data_embed = nn.Embedding(max_decoder_seq_len, self.hidden_dim)
        self.Decoders = [Decoder(dim_num = hidden_dim) for _ in range(decoder_num)]
        self.last_linear_layer = nn.Linear(self.hidden_dim, max_decoder_seq_len)

    def positional_encoding(self, position_max_length = 100):
        position = torch.arange(0, position_max_length, dtype = torch.float).unsqueeze(1)
        pe = torch.zeros(position_max_length, self.hidden_dim)
        div_term = torch.pow(torch.ones(self.hidden_dim // 2).fill_(10000), torch.arange(0, self.hidden_dim, 2) / torch.tensor(self.hidden_dim, dtype = torch.float32))
        pe[:, 0::2] = torch.sin(position/div_term)
        pe[:, 1::2] = torch.cos(position / div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

        return pe
    
    def forward(self, input, output, mask):
        input_embed = self.input_data_embed(input)
        input_embed += self.positional_encoding(self.max_encoder_seq_len)
        q, k, v = input_embed, input_embed, input_embed

        for encoder in self.Encoders:
            encoder_output = encoder(q, k, v)
            q = encoder_output
            k = encoder_output
            v = encoder_output

        output_embed = self.output_data_embed(output)
        output += self.positional_encoding(self.max_decoder_seq_len)
        output_embed = output_embed.masked_fill(mask.unsqueeze(-1), 0)
        d_q, d_k, d_v = output_embed, output_embed, output_embed

        for decoder in self.Decoders:
            decoder_output = decoder(d_q, d_k, d_v, encoder_output, mask)
            d_q = decoder_output
            d_k = decoder_output
            d_v = decoder_output

        output = F.softmax(self.last_linear_layer(decoder_output), dim = -1)
        return output